In [1]:
import sys
sys.path.append("../../")
sys.path.append("..")
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML

In [2]:
from cellmating.tracking._hausdorf_tracker import Tracker
from cellmating.visualize import animate_images
from cellmating.utils import file_traverse
from cellmating.io import imread, imsave
from cellmating.utils import crop_single_object
from cellmating.visualize import label2rgb

In [3]:
file_root = "/media/wd/T7/data/mating_all/20220903_YAV2104xYAV2105_003/tif/"
file_root = "/Volumes/T7/data/mating_all/20220903_YAV2104xYAV2105_003/tif/"
file_list = file_traverse(file_root, file_regular=".*._rigid.tif$")

In [39]:
filename = file_list[2]
print(filename)
image = imread(filename)
image = np.moveaxis(image, -1, 1)
print(image.shape)

/Volumes/T7/data/mating_all/20220903_YAV2104xYAV2105_003/tif/YAV2104xYAV2105_005_03_rigid.tif
(241, 4, 1020, 1020)


In [40]:
trace = Tracker(image[:, -1], max_miss=10, min_hist=3, low_ratio=0.6, high_ratio=1.5)
_ = trace()
traced_image = trace.to_image()

fusion: [[0 1 2]]
frame= 1 [1100] [1016, 1022, 2001]
divison: [[0 2 3]]
frame= 2 [1100] [1015, 1021, 2001]
frame= 3 [] [1006, 1022, 2001]
frame= 4 [1097] [1006, 1022, 2001]
frame= 5 [1046, 1097] [1021, 2001]
frame= 6 [1046, 1097] [1021, 2001]
frame= 7 [1097] [1015, 1021, 1062, 1083]
frame= 8 [1045, 1097] [1006, 1016, 1022, 1083]
frame= 9 [1097] [1006, 1016, 1022, 1084]
frame= 10 [1097] [1006, 1016, 1022, 2001]
divison: [[0 3 4]
 [2 6 7]]
frame= 11 [1097] [1006, 1016, 1022, 1084]
frame= 12 [] [1109]
divison: [[0 0 1]
 [1 2 3]]
frame= 13 [] [1110]
frame= 15 [] [1112]
frame= 16 [] [1132]
divison: [[0 0 2]]
frame= 17 [1047, 1102] [1006, 1018, 1025, 1125, 2001]
frame= 18 [1047] [1017, 1024, 1087, 1127, 2001]
frame= 19 [1047] [1006, 1018, 1025, 1127]
frame= 20 [1101] [1006, 1018, 1067, 1088, 1126]
frame= 21 [1101] [1018, 1025, 1067, 1089, 1129]
divison: [[0 1 2]]
frame= 22 [1101] [1006, 1019, 1026, 1105]
frame= 23 [1101] [1025, 1067, 1088, 1128]
divison: [[0 3 4]]
frame= 24 [1101] [1006, 101

In [41]:
annotation = label2rgb(traced_image)

In [44]:
# ani = animate_images(image[200:, -1,:,:])
# ani2 = animate_images(annotation[200:])

# html1 = HTML(ani.to_jshtml()).data
# html2 = HTML(ani2.to_jshtml()).data

# # Display the combined animations
# HTML(f"""
# <div style="display: flex;">
#     <div style="flex: 1; margin-right: 10px;">
#         {html1}
#     </div>
#     <div style="flex: 1; margin-left: 10px;">
#         {html2}
#     </div>
# </div>
# """)

In [43]:
imsave(filename[:-4]+"_trackv1.tif", traced_image, dtype=np.uint16, imagej=True)

In [24]:
from cellmating.utils import crop_object_to_dict
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
labels = np.unique(traced_image)[1:]
labels = labels[labels>2000]

In [37]:
import json

In [26]:
roi_dict = crop_object_to_dict(traced_image[-1], length=250, labels=labels)
with open(filename[:-4]+'.json', 'w') as json_file:
    json.dump(roi_dict, json_file, indent=4)

In [38]:


# # Sample dictionary
# data = {"name": "Alice", "age": 30, "city": "New York"}

# Save to a JSON file


In [29]:
with open('data.json', 'r') as json_file:
    loaded_data = json.load(json_file)
print(loaded_data)

{'2189': [770, 1020, 382, 632], '3142': [0, 250, 605, 855], '3165': [135, 385, 604, 854], '3171': [0, 250, 0, 250], '3185': [4, 254, 188, 438], '3186': [0, 250, 0, 250]}


In [28]:
import yaml

data = {"name": "Alice", "age": 30, "city": "New York"}
with open('data.yml', 'w') as yaml_file:
    yaml.dump(roi_dict, yaml_file, default_flow_style=False)

In [71]:
traced_image_stack = np.concatenate((image, traced_image[:,None,:,:]), axis=1)
imsave(filename[:-4]+"_track.tif", traced_image_stack, dtype=np.uint16, imagej=True)

KeyboardInterrupt: 

In [14]:
labels = np.unique(traced_image)[1:]
labels = labels[labels>2000]

In [15]:
for la in labels:
    
    print(la)
    x_0, x_1, y_0, y_1 = []

2189
3142
3165
3171
3185
3186


In [54]:
x_all = {}
y_all = {}
for la in labels:
    x, y = np.where(traced_image[-1] == la)
    x_all[la] = [x.min(), x.max(), y.min(), y.max()]
    # x_all.append([x.min(), x.max()])
    # y_all.append([y.min(), y.max()])
    

In [56]:
x_all[2189]

[923, 1002, 484, 531]

In [58]:
2*x.min() - (x.max() - x.min())
2*x.max() - x.min()

259

In [ ]:
import yaml

In [ ]:
import yaml
f = open('meta.yaml', 'w+')
yaml.dump(output_data, f, allow_unicode=True)